# Stage 2 Preprocessing

Membuang semua feature dengan data type string

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# Function Untuk Mengurangi size data
# Refer :- https://www.kaggle.com/rinnqd/reduce-memory-usage

def reduce_memory_usage(df):
  
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [3]:
# function untuk cek unique value
def check_unique_value(data):
    for i in data.columns:
        print(f"{[i]} \n {data[i].unique()} \n")

In [4]:
# function untuk cek persentase null value
def check_null_percentage(df):
    # Hitung total rows
    total_rows = len(df)

    # Hitung null values dan persentasenya
    null_counts = df.isnull().sum()
    null_percentage = (null_counts / total_rows * 100).round(2)

    # Buat dataframe hasil
    null_df = pd.DataFrame({
        'Total_Null': null_counts,
        'Null_Percentage': null_percentage
    })

    # Sorting berdasarkan persentase null terbesar
    null_df = null_df.sort_values('Null_Percentage', ascending=False)

    # Filter hanya kolom yang memiliki null values
    null_df = null_df[null_df['Total_Null'] > 0]

    return null_df

# Check null value (data) after handling null value
#null_analysis = check_null_percentage(data)
#print("Total rows in dataset:", len(data))
#print("\nNull Value Analysis:")
#print(null_analysis)

In [5]:
# function untuk melakukan One-Hot-Encoding
def one_hot_encode(df):
    original_columns = list(df.columns)
    categories = [cat for cat in df.columns if df[cat].dtype == 'object']
    df = pd.get_dummies(df, columns= categories, dummy_na= True) #one_hot_encode the categorical features
    categorical_columns = [cat for cat in df.columns if cat not in original_columns]
    return df, categorical_columns

# Importing Data

In [6]:
# import data
train = pd.read_csv("application_train.csv")
test  = pd.read_csv("application_test.csv")
bureau  = pd.read_csv("bureau.csv")
bureau_bal  = pd.read_csv("bureau_balance.csv")
prev  = pd.read_csv("previous_application.csv")
ccb  = pd.read_csv("credit_card_balance.csv")
pos_cb  = pd.read_csv("POS_CASH_balance.csv")
installment  = pd.read_csv("installments_payments.csv")

In [7]:
# check dimensions
print("Application:", train.shape, test.shape)
print("Bureau:", bureau.shape)
print("Bureau_balance:", bureau_bal.shape)
print("Previous_application:", prev.shape)
print("Credit_Card_Balance:", ccb.shape)
print("Posh_cash_Balance:", pos_cb.shape)
print("Instalment:", installment.shape)

Application: (307511, 122) (48744, 121)
Bureau: (1716428, 17)
Bureau_balance: (27299925, 3)
Previous_application: (1670214, 37)
Credit_Card_Balance: (3840312, 23)
Posh_cash_Balance: (10001358, 8)
Instalment: (13605401, 8)


# 1. Preprocessing Application Data

## 1.1 Application train

In [8]:
reduce_memory_usage(train)
train.info(verbose=True)

Memory usage of dataframe is 286.23 MB
Memory usage after optimization is: 92.38 MB
Decreased by 67.7%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 122 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    SK_ID_CURR                    int32  
 1    TARGET                        int8   
 2    NAME_CONTRACT_TYPE            object 
 3    CODE_GENDER                   object 
 4    FLAG_OWN_CAR                  object 
 5    FLAG_OWN_REALTY               object 
 6    CNT_CHILDREN                  int8   
 7    AMT_INCOME_TOTAL              float32
 8    AMT_CREDIT                    float32
 9    AMT_ANNUITY                   float32
 10   AMT_GOODS_PRICE               float32
 11   NAME_TYPE_SUITE               object 
 12   NAME_INCOME_TYPE              object 
 13   NAME_EDUCATION_TYPE           object 
 14   NAME_FAMILY_STATUS            object 
 15   NAME_HOUSING_TYPE             o

In [9]:
# List of common missing value strings
missing_values = [
    'NA', 'N/A', 'null', 'NaN','nan', 'XNA', 'None', 'empty', 'undefined', 'unknown', 'Unknown',
    '--', 'not available', '',' ','  ','   ', 'missing', 'unavailable', 'not applicable',
    'not found', 'no data', 'no value', 'n/a', 'none', 'null value', 'unknown value', 'blank', 'invalid', 'error',
    'unassigned', 'uninitialized', 'void', 'undisclosed', 'unreported', 'empty string',
    'na', 'not recorded', 'unanswered', 'not specified', 'unobserved', 'unpopulated',
    'none of the above', 'does not apply', 'ignored', 'unspecified', 'unfilled', 'inapplicable',
    'unavailable value', 'not entered', 'missing value', 'data not available', 'data missing',
    'not inputted', 'not given', 'unrecorded', 'unfurnished', 'undocumented', 'not listed',
    'unsupplied', 'empty field', 'unfilled field', 'unanswered question', 'no response',
    'not provided', 'not reported', 'unanswered', 'unacknowledged', 'no information',
    'no response recorded', 'no submission', 'no entry', 'not logged', 'unverified',
    'unobserved value', 'data absent', 'absence', 'unfilled cell', 'undefined data',
    'does not exist', 'not found', 'missing data', 'no answer', 'unavailable information',
    'no input', 'data error', 'unseen', 'unviewed', 'not selected', 'irrelevant',
    'skipped', 'no response given', 'no feedback', 'failed', 'discarded', 'no record',
    'unfetched', 'not completed', 'not finished', 'not updated', 'forgotten', 'rejected',
    'unsupplied data', 'data void', 'undetermined', 'not checked', 'not applicable here',
    'unattended', 'data missing', 'nonexistent', 'unsubscribed', 'denied', 'unaccepted',
    'unwitnessed', 'discarded entry', 'rejected data', 'unapplicable', 'non-populated',
    'unapplicable field', 'lost', 'absent data', 'no match', 'no result'
]

# Optimized function to replace missing-like values with NaN
def replace_missing_with_null(df, missing_values_set):
    # Select object type columns
    df_object_columns = df.select_dtypes(include=['object'])

    # Replace missing-like values with NaN
    for col in df_object_columns.columns:
        # Vectorized replacement of missing-like values with NaN
        df[col] = df[col].apply(lambda x: np.nan if str(x).strip() in missing_values_set else x)

    return df

In [10]:
train = replace_missing_with_null(train, missing_values)

In [11]:
# Check Null Value
null_analysis = check_null_percentage(train)
print("Total rows in dataset:", len(train))
print("\nNull Value Analysis:")
print(null_analysis)

Total rows in dataset: 307511

Null Value Analysis:
                              Total_Null  Null_Percentage
FONDKAPREMONT_MODE                215982            70.24
COMMONAREA_AVG                    214865            69.87
COMMONAREA_MODE                   214865            69.87
COMMONAREA_MEDI                   214865            69.87
NONLIVINGAPARTMENTS_MODE          213514            69.43
NONLIVINGAPARTMENTS_MEDI          213514            69.43
NONLIVINGAPARTMENTS_AVG           213514            69.43
LIVINGAPARTMENTS_AVG              210199            68.35
LIVINGAPARTMENTS_MEDI             210199            68.35
LIVINGAPARTMENTS_MODE             210199            68.35
FLOORSMIN_MODE                    208642            67.85
FLOORSMIN_AVG                     208642            67.85
FLOORSMIN_MEDI                    208642            67.85
YEARS_BUILD_AVG                   204488            66.50
YEARS_BUILD_MODE                  204488            66.50
YEARS_BUILD_MEDI    

Terdapat beberapa features yang memiliki null value (percentage) yang tinggi, sehingga saya memutuskan untuk me-drop beberapa feature yang memiliki null value di atas 40% dan dianggap tidak penting.

Untuk feature yang berada di atas 20% akan diisi dengan 0 atau unknown dan dibawah 20% akan diisi dengan median atau modus

In [12]:
train = train.drop(columns=['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'WALLSMATERIAL_MODE', 'HOUSETYPE_MODE',
                                        'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'EMERGENCYSTATE_MODE',
                                        'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'FONDKAPREMONT_MODE',
                                        'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'COMMONAREA_MODE','ELEVATORS_MODE', 'ENTRANCES_MODE', 
                                        'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 
                                        'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'TOTALAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE'], axis = 1)

In [13]:
def handling_na(data):
    # Mengisi nilai null untuk kolom tertentu dengan 0
    columns_fill_zero = ['COMMONAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'LIVINGAPARTMENTS_AVG', 
                         'FLOORSMIN_AVG', 'YEARS_BUILD_AVG', 'OWN_CAR_AGE', 
                         'LANDAREA_AVG', 'BASEMENTAREA_AVG', 'EXT_SOURCE_1', 
                         'NONLIVINGAREA_AVG', 'ELEVATORS_AVG', 'APARTMENTS_AVG', 
                         'ENTRANCES_AVG', 'LIVINGAREA_AVG', 'FLOORSMAX_AVG', 
                         'YEARS_BEGINEXPLUATATION_AVG', 'EXT_SOURCE_3']
    
    data[columns_fill_zero] = data[columns_fill_zero].fillna(0)  # Mengisi dengan 0

    # Mengisi nilai null untuk kolom tertentu dengan median
    columns_fill_median = ['AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_HOUR', 
                            'AMT_REQ_CREDIT_BUREAU_YEAR', 'AMT_REQ_CREDIT_BUREAU_MON', 
                            'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_WEEK', 
                            'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 
                            'DEF_60_CNT_SOCIAL_CIRCLE', 'OBS_30_CNT_SOCIAL_CIRCLE', 
                            'EXT_SOURCE_2', 'AMT_GOODS_PRICE', 'CNT_FAM_MEMBERS', 
                            'AMT_ANNUITY', 'DAYS_LAST_PHONE_CHANGE']
    
    data[columns_fill_median] = data[columns_fill_median].fillna(data[columns_fill_median].median())

    # Mengisi nilai null untuk kolom tertentu dengan modus
    columns_fill_mode = ['NAME_TYPE_SUITE', 'CODE_GENDER', 'NAME_FAMILY_STATUS', 'ORGANIZATION_TYPE']

    for column in columns_fill_mode:
        mode_value = data[column].mode()[0]
        data[column] = data[column].fillna(mode_value)

    # Mengisi nilai null menjadi unknown
    columns_fill_unknown = ['OCCUPATION_TYPE']

    data[columns_fill_unknown] = data[columns_fill_unknown].fillna('unkown')


In [14]:
handling_na(train)

In [15]:
# Check Duplicated Data
train.duplicated().sum()

np.int64(0)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 89 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   SK_ID_CURR                   307511 non-null  int32  
 1   TARGET                       307511 non-null  int8   
 2   NAME_CONTRACT_TYPE           307511 non-null  object 
 3   CODE_GENDER                  307511 non-null  object 
 4   FLAG_OWN_CAR                 307511 non-null  object 
 5   FLAG_OWN_REALTY              307511 non-null  object 
 6   CNT_CHILDREN                 307511 non-null  int8   
 7   AMT_INCOME_TOTAL             307511 non-null  float32
 8   AMT_CREDIT                   307511 non-null  float32
 9   AMT_ANNUITY                  307511 non-null  float32
 10  AMT_GOODS_PRICE              307511 non-null  float32
 11  NAME_TYPE_SUITE              307511 non-null  object 
 12  NAME_INCOME_TYPE             307511 non-null  object 
 13 

In [17]:
# Feature Encoding
def feature_encode(data):
    data['FLAG_OWN_CAR'] = data['FLAG_OWN_CAR'].map({'N': 0, 'Y': 1})
    data['FLAG_OWN_REALTY'] = data['FLAG_OWN_REALTY'].map({'N': 0, 'Y': 1})
    data['WEEKDAY_APPR_PROCESS_START'] = data['WEEKDAY_APPR_PROCESS_START'].map({'MONDAY': 1, 'TUESDAY': 2, 'WEDNESDAY': 3, 'THURSDAY': 4, 'FRIDAY': 5, 'SATURDAY': 6, 'SUNDAY': 7})
    data['NAME_EDUCATION_TYPE'] = data['NAME_EDUCATION_TYPE'].map({'Lower secondary': 0, #Sekolah Menengah Pertama
                                                                   'Secondary / secondary special': 1, #Sekolah Menengah Atas
                                                                   'Incomplete higher': 2, # Pendidikan Tinggi yang belum selesai
                                                                   'Higher education': 3, # Pendidikan Tinggi
                                                                   'Academic degree': 4}) # Gelar Akademik 

In [18]:
feature_encode(train)

In [19]:
testing1 = train['FLAG_OWN_CAR'].map({'N': 0, 'Y': 1})
testing1.value_counts()

Series([], Name: count, dtype: int64)

In [ ]:
# Feature Engineering

def application_FE(data):

    # Income Ratio
    data['DEBT_TO_INCOME'] = data['AMT_CREDIT'] / data['AMT_INCOME_TOTAL'] # Untuk mengukur seberapa mampu nasabah akan membayar dari total pendapatannya
    data["ANNUITY_TO_INCOME"] = data["AMT_ANNUITY"] / data["AMT_INCOME_TOTAL"] # Untuk mengukur rasio angsuran bulanan terhadap pendapatan nasabah

    # Usia dan Tenure
    data['AGE'] = round(-data['DAYS_BIRTH'] / 365).astype('Int64')
    data['TENURE'] = round(-data['DAYS_EMPLOYED'] / 365).astype('Int64') # Mengukur berapa tahun lamanya nasabah bekerja

    # Keluarga
    data['FAM_ADULT_MEMBERS'] = data['CNT_FAM_MEMBERS'] - data['CNT_CHILDREN'] # Menghitung berapa orang dewasa dalam satu keluarga
    data['CHILDREN_PER_FAM_MEMBERS'] = data['CNT_CHILDREN'] / data['CNT_FAM_MEMBERS'] # Menghitung rasio jumlah potensi tanggungan yang dimiliki

    # Kelengkapan Document
    data['NUM_DOCUMENTS'] = data.filter(like='FLAG_DOCUMENT').sum(axis=1)

    # External Source Mean
    data["EXT_SOURCE_MEAN"] = data[["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]].mean(axis = 1)


In [21]:
train_clean = train.copy()

In [22]:
application_FE(train_clean)

In [23]:
train_obj = train_clean.select_dtypes('object').columns
train_clean = train_clean.drop(columns=train_obj)

## 1.2 Application test

In [24]:
reduce_memory_usage(test)
test.info(verbose=True)

Memory usage of dataframe is 45.00 MB
Memory usage after optimization is: 14.60 MB
Decreased by 67.6%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Data columns (total 121 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    SK_ID_CURR                    int32  
 1    NAME_CONTRACT_TYPE            object 
 2    CODE_GENDER                   object 
 3    FLAG_OWN_CAR                  object 
 4    FLAG_OWN_REALTY               object 
 5    CNT_CHILDREN                  int8   
 6    AMT_INCOME_TOTAL              float32
 7    AMT_CREDIT                    float32
 8    AMT_ANNUITY                   float32
 9    AMT_GOODS_PRICE               float32
 10   NAME_TYPE_SUITE               object 
 11   NAME_INCOME_TYPE              object 
 12   NAME_EDUCATION_TYPE           object 
 13   NAME_FAMILY_STATUS            object 
 14   NAME_HOUSING_TYPE             object 
 15   REGION_POPULATION_RELATIVE    floa

In [25]:
test = replace_missing_with_null(test, missing_values)

In [26]:
# Check Null Value
null_analysis = check_null_percentage(test)
print("Total rows in dataset:", len(test))
print("\nNull Value Analysis:")
print(null_analysis)

Total rows in dataset: 48744

Null Value Analysis:
                              Total_Null  Null_Percentage
FONDKAPREMONT_MODE                 33710            69.16
COMMONAREA_MODE                    33495            68.72
COMMONAREA_MEDI                    33495            68.72
COMMONAREA_AVG                     33495            68.72
NONLIVINGAPARTMENTS_AVG            33347            68.41
NONLIVINGAPARTMENTS_MODE           33347            68.41
NONLIVINGAPARTMENTS_MEDI           33347            68.41
LIVINGAPARTMENTS_MEDI              32780            67.25
LIVINGAPARTMENTS_AVG               32780            67.25
LIVINGAPARTMENTS_MODE              32780            67.25
FLOORSMIN_MODE                     32466            66.61
FLOORSMIN_AVG                      32466            66.61
FLOORSMIN_MEDI                     32466            66.61
OWN_CAR_AGE                        32312            66.29
YEARS_BUILD_AVG                    31818            65.28
YEARS_BUILD_MODE     

In [27]:
test = test.drop(columns=['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI', 'WALLSMATERIAL_MODE', 'HOUSETYPE_MODE',
                                        'FLOORSMAX_MEDI', 'FLOORSMIN_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'EMERGENCYSTATE_MODE',
                                        'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI','YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI', 'FONDKAPREMONT_MODE',
                                        'APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'COMMONAREA_MODE','ELEVATORS_MODE', 'ENTRANCES_MODE', 
                                        'FLOORSMAX_MODE', 'FLOORSMIN_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 
                                        'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE', 'TOTALAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE', 'YEARS_BUILD_MODE'], axis = 1)

In [28]:
handling_na(test)

In [29]:
# Check Duplicated Data
test.duplicated().sum()

np.int64(0)

In [30]:
# Feature Encoding
feature_encode(test)

In [31]:
# Feature Engineering

application_FE(test)

In [32]:
test_clean = test.copy()

In [33]:
column_obj = test_clean.select_dtypes('object').columns
test_clean = test_clean.drop(columns=column_obj)

In [34]:
print('Train dimension:', train_clean.shape)
print('Test dimension:', test_clean.shape)

Train dimension: (307511, 89)
Test dimension: (48744, 88)


# 2. Preprocessing Bureau Data

## 2.1 Bureau Balance

In [35]:
reduce_memory_usage(bureau_bal)
bureau_bal.info()

Memory usage of dataframe is 624.85 MB
Memory usage after optimization is: 338.46 MB
Decreased by 45.8%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27299925 entries, 0 to 27299924
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   SK_ID_BUREAU    int32 
 1   MONTHS_BALANCE  int8  
 2   STATUS          object
dtypes: int32(1), int8(1), object(1)
memory usage: 338.5+ MB


In [36]:
# Aggregat Data
average_status = bureau_bal.groupby(['SK_ID_BUREAU','STATUS'])['MONTHS_BALANCE'].mean().reset_index()

In [37]:
# Check Null Value
null_analysis = check_null_percentage(average_status)
print("Total rows in dataset:", len(average_status))
print("\nNull Value Analysis:")
print(null_analysis)

Total rows in dataset: 1714229

Null Value Analysis:
Empty DataFrame
Columns: [Total_Null, Null_Percentage]
Index: []


In [38]:
print('Jumlah data duplikat pada Bureau Balance: ', average_status.duplicated().sum())

Jumlah data duplikat pada Bureau Balance:  0


## 2.2 Bureau

In [39]:
reduce_memory_usage(bureau)
bureau.info()

Memory usage of dataframe is 222.62 MB
Memory usage after optimization is: 112.95 MB
Decreased by 49.3%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int32  
 1   SK_ID_BUREAU            int32  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int16  
 5   CREDIT_DAY_OVERDUE      int16  
 6   DAYS_CREDIT_ENDDATE     float16
 7   DAYS_ENDDATE_FACT       float16
 8   AMT_CREDIT_MAX_OVERDUE  float32
 9   CNT_CREDIT_PROLONG      int8   
 10  AMT_CREDIT_SUM          float32
 11  AMT_CREDIT_SUM_DEBT     float32
 12  AMT_CREDIT_SUM_LIMIT    float32
 13  AMT_CREDIT_SUM_OVERDUE  float32
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int32  
 16  AMT_ANNUITY             float32
dtypes: float16(2), float32(6), int16(2), int32(3), int8(1), object(3)
memory us

In [40]:
bureau['SK_ID_BUREAU'].nunique() # Primary Key

1716428

In [41]:
# Menggabungkan bureau dan bureau balance
bureau_merge = pd.merge(bureau, average_status, on='SK_ID_BUREAU', how='left')
print('bureau_merge dimension:', bureau_merge.shape)
bureau_merge.sample()

bureau_merge dimension: (2573172, 19)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,STATUS,MONTHS_BALANCE
627662,261731,6131921,Active,currency 1,-325,0,1501.0,NaN,NaN,0,900000.0,NaN,NaN,0.0,Consumer credit,-15,2867.715088,X,-8.0


In [42]:
bureau_merge = replace_missing_with_null(bureau_merge, missing_values)

In [43]:
# Check Null Value
null_analysis = check_null_percentage(bureau_merge)
print("Total rows in dataset:", len(bureau_merge))
print("\nNull Value Analysis:")
print(null_analysis)

Total rows in dataset: 2573172

Null Value Analysis:
                        Total_Null  Null_Percentage
AMT_CREDIT_MAX_OVERDUE     1724222            67.01
AMT_ANNUITY                1502714            58.40
STATUS                      942074            36.61
MONTHS_BALANCE              942074            36.61
AMT_CREDIT_SUM_LIMIT        915397            35.57
DAYS_ENDDATE_FACT           879273            34.17
AMT_CREDIT_SUM_DEBT         400264            15.56
DAYS_CREDIT_ENDDATE         156975             6.10
AMT_CREDIT_SUM                  13             0.00


Feature - feature yang memiliki null value di atas 40% terlihat cukup penting sehingga saya akan mengubah null value tersebut menjadi 0 dan untuk feature STATUS akan di drop baris.

In [44]:
def handling_na_bureau(data):
    # Mengisi nilai null menjadi 0
    columns_fill_zero = ['AMT_CREDIT_MAX_OVERDUE', 'AMT_ANNUITY']
    
    data[columns_fill_zero] = data[columns_fill_zero].fillna(0)

    # Mengisi nilai null menjadi median
    columns_fill_median = ['AMT_CREDIT_SUM_LIMIT', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_SUM_DEBT', 'DAYS_CREDIT_ENDDATE', 'AMT_CREDIT_SUM', 'MONTHS_BALANCE']
    
    data[columns_fill_median] = data[columns_fill_median].fillna(data[columns_fill_median].median())

    # Mengisi nilai null menjadi unkown
    columns_fill_unknown = ['STATUS']

    data['STATUS'] = data['STATUS'].fillna('unkown')

In [45]:
handling_na_bureau(bureau_merge)

In [46]:
print('Jumlah data duplikat pada Bureau_merge: ', bureau_merge.duplicated().sum())

Jumlah data duplikat pada Bureau_merge:  0


In [47]:
bureau_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2573172 entries, 0 to 2573171
Data columns (total 19 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int32  
 1   SK_ID_BUREAU            int32  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int16  
 5   CREDIT_DAY_OVERDUE      int16  
 6   DAYS_CREDIT_ENDDATE     float16
 7   DAYS_ENDDATE_FACT       float16
 8   AMT_CREDIT_MAX_OVERDUE  float32
 9   CNT_CREDIT_PROLONG      int8   
 10  AMT_CREDIT_SUM          float32
 11  AMT_CREDIT_SUM_DEBT     float32
 12  AMT_CREDIT_SUM_LIMIT    float32
 13  AMT_CREDIT_SUM_OVERDUE  float32
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int32  
 16  AMT_ANNUITY             float32
 17  STATUS                  object 
 18  MONTHS_BALANCE          float64
dtypes: float16(2), float32(6), float64(1), int16(2), int32(3), int8(1), object(4)
memory usage: 208.6+ MB


In [48]:
# Feature Encoding
def bureau_merge_encoding(data):
    bureau_merge['CREDIT_CURRENCY'] = bureau_merge['CREDIT_CURRENCY'].map({'currency 1': 1, 'currency 2': 2, 'currency 3': 3, 'currency 4': 4})

In [49]:
bureau_merge_encoding(bureau_merge)

In [50]:
# Feature Engineering
def bureau_merge_FE(data):

    # Durasi lamanya pinjaman
    data['DAYS_DURATION_CREDIT'] = data['DAYS_ENDDATE_FACT'] - data['DAYS_CREDIT']


In [51]:
bureau_merge_FE(bureau_merge)

In [52]:
column_obj = bureau_merge.select_dtypes('object').columns
bureau_merge = bureau_merge.drop(columns=column_obj)

In [53]:
def aggregate_bureau(df, group_by='SK_ID_CURR'):
    
    agg_bureau = df.groupby(group_by).agg({
        # Jumlah kredit
        'SK_ID_BUREAU': 'count',  # Jumlah kredit yang dimiliki
        
        # Agregasi waktu kredit
        'DAYS_CREDIT': ['min', 'max', 'mean'],  # Rentang dan rata-rata usia kredit
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],  # Rentang dan rata-rata tanggal berakhir
        'DAYS_ENDDATE_FACT': ['min', 'max', 'mean'],  # Rentang dan rata-rata tanggal aktual berakhir
        'DAYS_CREDIT_UPDATE': ['min', 'max', 'mean'],  # Rentang dan rata-rata update terakhir
        'DAYS_DURATION_CREDIT': ['min', 'max', 'mean'], # Rentang dan rata-rata lama pinjaman terjadi
        # Agregasi jumlah hari overdue
        'CREDIT_DAY_OVERDUE': ['max', 'mean', 'sum'],  # Maksimum, rata-rata, dan total hari keterlambatan
        
        # Agregasi nilai kredit
        'AMT_CREDIT_MAX_OVERDUE': ['max', 'mean'],  # Maksimum dan rata-rata nilai overdue
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],  # Statistik total kredit
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],  # Statistik total hutang
        'AMT_CREDIT_SUM_LIMIT': ['max', 'mean', 'sum'],  # Statistik limit kredit
        'AMT_CREDIT_SUM_OVERDUE': ['max', 'mean', 'sum'],  # Statistik total overdue
        'AMT_ANNUITY': ['max', 'mean', 'sum'],  # Statistik annuity
        
        # Agregasi lainnya
        'CNT_CREDIT_PROLONG': ['max', 'sum'],  # Maksimum dan total perpanjangan kredit
        'MONTHS_BALANCE': ['min', 'max', 'mean']  # Statistik balance bulanan
    })
    
    # Flatten kolom multiindex
    agg_bureau.columns = ['BUR_' + '_'.join(col).upper() for col in agg_bureau.columns]
    
    return agg_bureau

In [54]:
bureau_merge = aggregate_bureau(bureau_merge, group_by='SK_ID_CURR')

In [55]:
#bureau_merge.columns = ['BUR_' + ''.join(col).upper() for col in bureau_merge.columns]

In [56]:
bureau_merge.head()

,BUR_SK_ID_BUREAU_COUNT,BUR_DAYS_CREDIT_MIN,BUR_DAYS_CREDIT_MAX,BUR_DAYS_CREDIT_MEAN,BUR_DAYS_CREDIT_ENDDATE_MIN,BUR_DAYS_CREDIT_ENDDATE_MAX,BUR_DAYS_CREDIT_ENDDATE_MEAN,BUR_DAYS_ENDDATE_FACT_MIN,BUR_DAYS_ENDDATE_FACT_MAX,BUR_DAYS_ENDDATE_FACT_MEAN,BUR_DAYS_CREDIT_UPDATE_MIN,BUR_DAYS_CREDIT_UPDATE_MAX,BUR_DAYS_CREDIT_UPDATE_MEAN,BUR_DAYS_DURATION_CREDIT_MIN,BUR_DAYS_DURATION_CREDIT_MAX,BUR_DAYS_DURATION_CREDIT_MEAN,BUR_CREDIT_DAY_OVERDUE_MAX,BUR_CREDIT_DAY_OVERDUE_MEAN,BUR_CREDIT_DAY_OVERDUE_SUM,BUR_AMT_CREDIT_MAX_OVERDUE_MAX,BUR_AMT_CREDIT_MAX_OVERDUE_MEAN,BUR_AMT_CREDIT_SUM_MAX,BUR_AMT_CREDIT_SUM_MEAN,BUR_AMT_CREDIT_SUM_SUM,BUR_AMT_CREDIT_SUM_DEBT_MAX,BUR_AMT_CREDIT_SUM_DEBT_MEAN,BUR_AMT_CREDIT_SUM_DEBT_SUM,BUR_AMT_CREDIT_SUM_LIMIT_MAX,BUR_AMT_CREDIT_SUM_LIMIT_MEAN,BUR_AMT_CREDIT_SUM_LIMIT_SUM,BUR_AMT_CREDIT_SUM_OVERDUE_MAX,BUR_AMT_CREDIT_SUM_OVERDUE_MEAN,BUR_AMT_CREDIT_SUM_OVERDUE_SUM,BUR_AMT_ANNUITY_MAX,BUR_AMT_ANNUITY_MEAN,BUR_AMT_ANNUITY_SUM,BUR_CNT_CREDIT_PROLONG_MAX,BUR_CNT_CREDIT_PROLONG_SUM,BUR_MONTHS_BALANCE_MIN,BUR_MONTHS_BALANCE_MAX,BUR_MONTHS_BALANCE_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,17,-1572,-49,-813.941176,-1329.0,1778.0,-40.588234,-1328.0,-544.0,-851.588257,-155,-6,-103.823529,-859.0,335.0,-37.647060,0,0.0,0,0.00000,0.000000,378000.000000,197981.468750,3.365685e+06,373239.0,70368.085938,1196257.5,0.000000,0.000000,0.000000,0.0,0.0,0.0,10822.5,2641.235352,44901.0,0,0,-47.714286,0.0,-17.248599
100002,26,-1437,-103,-981.538462,-1072.0,780.0,-388.384613,-1185.0,-36.0,-846.923096,-1185,-7,-607.153846,-805.0,609.0,134.615387,0,0.0,0,5043.64502,902.011108,450000.000000,121337.500000,3.154775e+06,245781.0,37812.460938,983124.0,31988.564453,2460.658691,63977.128906,0.0,0.0,0.0,0.0,0.000000,0.0,0,0,-44.000000,-0.5,-24.455128
100003,4,-2586,-606,-1400.750000,-2434.0,1216.0,-544.500000,-2132.0,-540.0,-1050.250000,-2131,-43,-816.000000,-302.0,1096.0,350.500000,0,0.0,0,0.00000,0.000000,810000.000000,254350.125000,1.017400e+06,0.0,0.000000,0.0,810000.000000,202500.000000,810000.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,0,-20.000000,-20.0,-20.000000
100004,2,-1326,-408,-867.000000,-595.0,-382.0,-488.500000,-683.0,-382.0,-532.500000,-682,-382,-532.000000,26.0,643.0,334.500000,0,0.0,0,0.00000,0.000000,94537.796875,94518.898438,1.890378e+05,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0,0,-20.000000,-20.0,-20.000000
100005,6,-373,-62,-230.000000,-128.0,1324.0,197.333328,-908.0,-123.0,-515.500000,-121,-11,-72.666667,-846.0,250.0,-285.500000,0,0.0,0,0.00000,0.000000,568800.000000,133992.000000,8.039520e+05,543087.0,98955.000000,593730.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,4261.5,1420.500000,8523.0,0,0,-12.000000,0.0,-4.250000


In [57]:
bureau_clean = bureau_merge.copy()

In [58]:
print('Train dimension:', train_clean.shape)
print('Test dimension:', test_clean.shape)
print('Bureau & Bureau Balance dimension:', bureau_clean.shape)

Train dimension: (307511, 89)
Test dimension: (48744, 88)
Bureau & Bureau Balance dimension: (305811, 41)


# 3. Previous Loan Data

## 3.1 Installment

Dataset installment memberikan detail tentang waktu dan jumlah angsuran sebelumnya sehingga berguna untuk memahami perilaku pembayaran klien di masa lampau yang dapat menjadi prediktor kuat untuk kinerja pembayaran pinjaman di masa yang akan datang.

In [59]:
reduce_memory_usage(installment)
installment.info()

Memory usage of dataframe is 830.41 MB
Memory usage after optimization is: 311.40 MB
Decreased by 62.5%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int32  
 1   SK_ID_CURR              int32  
 2   NUM_INSTALMENT_VERSION  float16
 3   NUM_INSTALMENT_NUMBER   int16  
 4   DAYS_INSTALMENT         float16
 5   DAYS_ENTRY_PAYMENT      float16
 6   AMT_INSTALMENT          float32
 7   AMT_PAYMENT             float32
dtypes: float16(3), float32(2), int16(1), int32(2)
memory usage: 311.4 MB


In [60]:
installment = replace_missing_with_null(installment, missing_values)

In [61]:
# Check Null Value
null_analysis = check_null_percentage(installment)
print("Total rows in dataset:", len(installment))
print("\nNull Value Analysis:")
print(null_analysis)

Total rows in dataset: 13605401

Null Value Analysis:
                    Total_Null  Null_Percentage
AMT_PAYMENT               2905             0.02
DAYS_ENTRY_PAYMENT        2905             0.02


Karena Null percentage dibawah 5%, saya memutuskan mengubah null value menjadi median.

In [62]:
def handling_na_installment(data):
    # Membuang nilai null
    columns_fill_median = ['AMT_PAYMENT', 'DAYS_ENTRY_PAYMENT']
    data[columns_fill_median] = data[columns_fill_median].fillna(data[columns_fill_median].median())

In [63]:
handling_na_installment(installment)

In [64]:
installment.drop_duplicates(inplace=True)
print('Jumlah data duplikat: ',installment.duplicated().sum())

Jumlah data duplikat:  0


In [65]:
installment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13605386 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int32  
 1   SK_ID_CURR              int32  
 2   NUM_INSTALMENT_VERSION  float16
 3   NUM_INSTALMENT_NUMBER   int16  
 4   DAYS_INSTALMENT         float16
 5   DAYS_ENTRY_PAYMENT      float16
 6   AMT_INSTALMENT          float32
 7   AMT_PAYMENT             float32
dtypes: float16(3), float32(2), int16(1), int32(2)
memory usage: 415.2 MB


In [66]:
# Feature Engineering
def installment_FE(data):
    # Melihat keterlambatan pembayaran
    data['DAYS_LATE'] = data['DAYS_ENTRY_PAYMENT'] - data['DAYS_INSTALMENT'] # Nilai positif menunjukan keterlambatan pembayaran

    # Ratio pembayaran yang diharapkan vs aktual
    data['PAYMENT_RATIO'] = data['AMT_PAYMENT']/data['AMT_INSTALMENT'] # Nilai kurang dari satu menunjukan pembayaran yang kurang

    # Melihat Pembayaran yang Kurang
    data['PAYMENT_DIFF'] = data['AMT_INSTALMENT'] - data['AMT_PAYMENT'] # Jika hasilnya positif menunjukan bahwa ada pembayaran yang kurang

    # Pembagian waktu
    data['MONTH_INSTALMENT'] = round(data['DAYS_INSTALMENT'] / 12)
    data['MONTH_ENTRY_PAYMENT'] = round(data['DAYS_ENTRY_PAYMENT'] / 12)
    data['YEAR_INSTALLMENT'] = round(data['DAYS_INSTALMENT'] / 365)
    data['YEAR_ENTRY_PAYMENT'] = round(data['DAYS_ENTRY_PAYMENT'] / 365)

    # Flag
    data['FLAG_LATE_PAYMENT'] = (data['DAYS_LATE'] > 0).astype(int) # Jika days_late atau month_installment lebih kecil dibandingkan month_entry_payment menunjukan bahwa pembayaran aktual telat dari yang seharusnya (installment)
    data['FLAG_UNDERPAYMENT'] = (data['PAYMENT_RATIO'] > 1).astype(int)

In [67]:
installment_FE(installment)

In [68]:
column_obj = installment.select_dtypes('object').columns
installment = installment.drop(columns=column_obj)

In [69]:
def aggregate_installment(df, group_by='SK_ID_CURR'):
    agg_installment = df.groupby(group_by).agg({
        'NUM_INSTALMENT_VERSION': ['nunique', 'mean'],  # Versi cicilan
        'NUM_INSTALMENT_NUMBER': ['min', 'max', 'mean', 'sum'],  # Nomor cicilan
        
        # Agregasi keterlambatan
        'DAYS_LATE': ['mean', 'max', 'sum', 'count'],  # Rata-rata keterlambatan, keterlambatan maksimal, total keterlambatan
        'FLAG_LATE_PAYMENT': ['mean', 'sum'],  # % keterlambatan, jumlah keterlambatan
        
        # Agregasi pembayaran
        'PAYMENT_RATIO': ['mean', 'min'],  # Rata-rata ratio pembayaran, ratio pembayaran terendah
        'PAYMENT_DIFF': ['mean', 'max', 'sum'],  # Rata-rata selisih pembayaran, selisih terbesar, total selisih
        'FLAG_UNDERPAYMENT': ['mean', 'sum'],  # % underpayment, jumlah underpayment
        
        # Agregasi nilai installment dan payment
        'AMT_INSTALMENT': ['mean', 'max', 'sum'],  # Rata-rata installment, installment terbesar, total installment
        'AMT_PAYMENT': ['mean', 'max', 'sum'],  # Rata-rata pembayaran, pembayaran terbesar, total pembayaran
        
        # Agregasi waktu
        'MONTH_INSTALMENT': ['min', 'max'],  # Bulan pertama dan terakhir installment
        'MONTH_ENTRY_PAYMENT': ['min', 'max'],  # Bulan pertama dan terakhir pembayaran
        'YEAR_INSTALLMENT': ['min', 'max'],  # Tahun pertama dan terakhir installment
        'YEAR_ENTRY_PAYMENT': ['min', 'max']  # Tahun pertama dan terakhir pembayaran
    })
    
    # Flatten kolom multiindex
    agg_installment.columns = ['INST_' + '_'.join(col).upper() for col in agg_installment.columns]
    
    return agg_installment

In [70]:
installment = aggregate_installment(installment, group_by='SK_ID_CURR')

In [71]:
# installment.columns = ['INST_' + ''.join(col).upper() for col in installment.columns]

In [72]:
installment.head()

,INST_NUM_INSTALMENT_VERSION_NUNIQUE,INST_NUM_INSTALMENT_VERSION_MEAN,INST_NUM_INSTALMENT_NUMBER_MIN,INST_NUM_INSTALMENT_NUMBER_MAX,INST_NUM_INSTALMENT_NUMBER_MEAN,INST_NUM_INSTALMENT_NUMBER_SUM,INST_DAYS_LATE_MEAN,INST_DAYS_LATE_MAX,INST_DAYS_LATE_SUM,INST_DAYS_LATE_COUNT,INST_FLAG_LATE_PAYMENT_MEAN,INST_FLAG_LATE_PAYMENT_SUM,INST_PAYMENT_RATIO_MEAN,INST_PAYMENT_RATIO_MIN,INST_PAYMENT_DIFF_MEAN,INST_PAYMENT_DIFF_MAX,INST_PAYMENT_DIFF_SUM,INST_FLAG_UNDERPAYMENT_MEAN,INST_FLAG_UNDERPAYMENT_SUM,INST_AMT_INSTALMENT_MEAN,INST_AMT_INSTALMENT_MAX,INST_AMT_INSTALMENT_SUM,INST_AMT_PAYMENT_MEAN,INST_AMT_PAYMENT_MAX,INST_AMT_PAYMENT_SUM,INST_MONTH_INSTALMENT_MIN,INST_MONTH_INSTALMENT_MAX,INST_MONTH_ENTRY_PAYMENT_MIN,INST_MONTH_ENTRY_PAYMENT_MAX,INST_YEAR_INSTALLMENT_MIN,INST_YEAR_INSTALLMENT_MAX,INST_YEAR_ENTRY_PAYMENT_MIN,INST_YEAR_ENTRY_PAYMENT_MAX
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,2,1.142857,1,4,2.714286,19,-7.428571,10.0,-52.0,7,0.142857,1,1.0,1.0,0.0,0.0,0.0,0.0,0,5885.132324,17397.900391,4.119593e+04,5885.132324,17397.900391,4.119593e+04,-243.0,-135.0,-243.0,-136.0,-8.0,-4.0,-8.0,-4.0
100002,2,1.052632,1,19,10.000000,190,-20.421053,-12.0,-388.0,19,0.000000,0,1.0,1.0,0.0,0.0,0.0,0.0,0,11559.247070,53093.746094,2.196257e+05,11559.247070,53093.746094,2.196257e+05,-47.0,-2.0,-49.0,-4.0,-2.0,-0.0,-2.0,-0.0
100003,2,1.040000,1,12,5.080000,127,-7.200000,-2.0,-180.0,25,0.000000,0,1.0,1.0,0.0,0.0,0.0,0.0,0,64754.585938,560835.375000,1.618865e+06,64754.585938,560835.375000,1.618865e+06,-192.0,-45.0,-194.0,-45.0,-6.0,-1.0,-6.0,-1.0
100004,2,1.333333,1,3,2.000000,6,-7.666667,-3.0,-23.0,3,0.000000,0,1.0,1.0,0.0,0.0,0.0,0.0,0,7096.154785,10573.964844,2.128846e+04,7096.154785,10573.964844,2.128846e+04,-65.0,-60.0,-66.0,-61.0,-2.0,-2.0,-2.0,-2.0
100005,2,1.111111,1,9,5.000000,45,-23.555555,1.0,-212.0,9,0.111111,1,1.0,1.0,0.0,0.0,0.0,0.0,0,6240.205078,17656.244141,5.616184e+04,6240.205078,17656.244141,5.616184e+04,-59.0,-39.0,-61.0,-39.0,-2.0,-1.0,-2.0,-1.0


In [73]:
installment_clean = installment.copy()

In [74]:
print('Train dimension:', train_clean.shape)
print('Test dimension:', test_clean.shape)
print('Bureau & Bureau Balance dimension:', bureau_clean.shape)
print('Instalment dimension: ', installment_clean.shape)

Train dimension: (307511, 89)
Test dimension: (48744, 88)
Bureau & Bureau Balance dimension: (305811, 41)
Instalment dimension:  (339587, 33)


## 3.2 Credit Card Balance

In [75]:
reduce_memory_usage(ccb)
ccb.info()

Memory usage of dataframe is 673.88 MB
Memory usage after optimization is: 289.33 MB
Decreased by 57.1%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 23 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   SK_ID_PREV                  int32  
 1   SK_ID_CURR                  int32  
 2   MONTHS_BALANCE              int8   
 3   AMT_BALANCE                 float32
 4   AMT_CREDIT_LIMIT_ACTUAL     int32  
 5   AMT_DRAWINGS_ATM_CURRENT    float32
 6   AMT_DRAWINGS_CURRENT        float32
 7   AMT_DRAWINGS_OTHER_CURRENT  float32
 8   AMT_DRAWINGS_POS_CURRENT    float32
 9   AMT_INST_MIN_REGULARITY     float32
 10  AMT_PAYMENT_CURRENT         float32
 11  AMT_PAYMENT_TOTAL_CURRENT   float32
 12  AMT_RECEIVABLE_PRINCIPAL    float32
 13  AMT_RECIVABLE               float32
 14  AMT_TOTAL_RECEIVABLE        float32
 15  CNT_DRAWINGS_ATM_CURRENT    float16
 16  CNT_DRAWINGS_CURRENT        int16  
 17 

In [76]:
ccb = replace_missing_with_null(ccb, missing_values)

In [77]:
# Check Null Value
null_analysis = check_null_percentage(ccb)
print("Total rows in dataset:", len(ccb))
print("\nNull Value Analysis:")
print(null_analysis)

Total rows in dataset: 3840312

Null Value Analysis:
                            Total_Null  Null_Percentage
AMT_PAYMENT_CURRENT             767988            20.00
CNT_DRAWINGS_POS_CURRENT        749816            19.52
AMT_DRAWINGS_ATM_CURRENT        749816            19.52
CNT_DRAWINGS_ATM_CURRENT        749816            19.52
AMT_DRAWINGS_POS_CURRENT        749816            19.52
AMT_DRAWINGS_OTHER_CURRENT      749816            19.52
CNT_DRAWINGS_OTHER_CURRENT      749816            19.52
CNT_INSTALMENT_MATURE_CUM       305236             7.95
AMT_INST_MIN_REGULARITY         305236             7.95


In [78]:
def handling_na_ccb(data):
    # Mengisi nilai null dengan median
    columns_fill_median = ['AMT_PAYMENT_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 
                            'AMT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_ATM_CURRENT', 
                            'AMT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 
                            'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 
                            'AMT_INST_MIN_REGULARITY']
    
    data[columns_fill_median] = data[columns_fill_median].fillna(data[columns_fill_median].median())

In [79]:
handling_na_ccb(ccb)

In [80]:
ccb.duplicated().sum()

np.int64(0)

In [81]:
# Feature Engineering
def credit_card_balance_FE(data):
    
    # Ratio limit kredit terpakai
    data['LIMIT_RATIO'] = data['AMT_BALANCE'] / data['AMT_CREDIT_LIMIT_ACTUAL']

    # Total Penarikan dari Semua Channel
    data['TOTAL_DRAWINGS'] = (data['AMT_DRAWINGS_ATM_CURRENT'] + 
                              data['AMT_DRAWINGS_CURRENT'] +
                              data['AMT_DRAWINGS_OTHER_CURRENT'] +
                              data['AMT_DRAWINGS_POS_CURRENT'])
    
    # Total Frekuensi Penarikan dari Semua Channel
    data['TOTAL_DRAWINGS_COUNT'] = (data['CNT_DRAWINGS_ATM_CURRENT'] +
                                    data['CNT_DRAWINGS_CURRENT'] +
                                    data['CNT_DRAWINGS_OTHER_CURRENT'] +
                                    data['CNT_DRAWINGS_POS_CURRENT'])
    
    # Rata - Rata Jumlah Uang yang ditarik per Penarikan
    data['AVG_DRAWING'] = data['TOTAL_DRAWINGS'] / data['TOTAL_DRAWINGS_COUNT']

    # Percentase Penarikan di setiap Channel
    data['DRAWINGS_ATM_PERCENTAGE'] = data['AMT_DRAWINGS_ATM_CURRENT'] / data['TOTAL_DRAWINGS']
    data['DRAWINGS_POS_PERCENTAGE'] = data['AMT_DRAWINGS_POS_CURRENT'] / data['TOTAL_DRAWINGS']
    data['DRAWINGS_OTHER_PERCENTAGE'] = data['AMT_DRAWINGS_OTHER_CURRENT'] / data['TOTAL_DRAWINGS']

    # Flag Pembayaran yang Telat
    data['FLAG_LATE_PAYMENT'] = (data['NAME_CONTRACT_STATUS'] == 'Late').astype(int)

    # Handling infinite value
    data = data.replace([np.inf, -np.inf], np.nan)



In [82]:
credit_card_balance_FE(ccb)

In [83]:
column_obj = ccb.select_dtypes('object').columns
ccb = ccb.drop(columns=column_obj)

In [84]:
ccb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 30 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   SK_ID_PREV                  int32  
 1   SK_ID_CURR                  int32  
 2   MONTHS_BALANCE              int8   
 3   AMT_BALANCE                 float32
 4   AMT_CREDIT_LIMIT_ACTUAL     int32  
 5   AMT_DRAWINGS_ATM_CURRENT    float32
 6   AMT_DRAWINGS_CURRENT        float32
 7   AMT_DRAWINGS_OTHER_CURRENT  float32
 8   AMT_DRAWINGS_POS_CURRENT    float32
 9   AMT_INST_MIN_REGULARITY     float32
 10  AMT_PAYMENT_CURRENT         float32
 11  AMT_PAYMENT_TOTAL_CURRENT   float32
 12  AMT_RECEIVABLE_PRINCIPAL    float32
 13  AMT_RECIVABLE               float32
 14  AMT_TOTAL_RECEIVABLE        float32
 15  CNT_DRAWINGS_ATM_CURRENT    float16
 16  CNT_DRAWINGS_CURRENT        int16  
 17  CNT_DRAWINGS_OTHER_CURRENT  float16
 18  CNT_DRAWINGS_POS_CURRENT    float16
 19  CNT_INSTALMENT_MATURE

In [85]:
# Aggregate 
def aggregate_ccb(data):
    agg_ccb = data.groupby('SK_ID_CURR').agg({
        'MONTHS_BALANCE': 'count',                     
        'AMT_BALANCE': ['min', 'max', 'mean', 'sum'], 
        'AMT_CREDIT_LIMIT_ACTUAL': ['min', 'max', 'mean', 'sum'],
        'AMT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'mean', 'sum'],
        'AMT_DRAWINGS_CURRENT': ['min', 'max', 'mean', 'sum'], 
        'AMT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'mean', 'sum'],   
        'AMT_INST_MIN_REGULARITY': ['min', 'max', 'mean', 'sum'],
        'AMT_PAYMENT_TOTAL_CURRENT': ['min', 'max', 'mean', 'sum'],
        'AMT_RECEIVABLE_PRINCIPAL': ['min', 'max', 'mean', 'sum'],
        'AMT_RECIVABLE': ['min', 'max', 'mean', 'sum'],
        'AMT_TOTAL_RECEIVABLE': ['min', 'max', 'mean', 'sum'],
        'CNT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'mean', 'sum'],    
        'AMT_DRAWINGS_POS_CURRENT': ['min', 'max', 'mean', 'sum'],         
        'AMT_PAYMENT_CURRENT': ['min', 'max', 'mean', 'sum'],
        'CNT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'mean', 'sum'],             
        'CNT_DRAWINGS_CURRENT': ['min', 'max', 'mean', 'sum'],
        'CNT_DRAWINGS_POS_CURRENT': ['min', 'max', 'mean', 'sum'],
        'CNT_INSTALMENT_MATURE_CUM': ['min', 'max', 'mean', 'sum'],         
        'SK_DPD': ['min', 'max', 'mean', 'sum'],                    
        'SK_DPD_DEF': ['min', 'max', 'mean', 'sum'],
        'TOTAL_DRAWINGS': ['min', 'max', 'mean', 'sum'],
        'TOTAL_DRAWINGS_COUNT': ['min', 'max', 'mean', 'sum'],
        'AVG_DRAWING': ['min', 'max', 'mean', 'sum'],
        'DRAWINGS_ATM_PERCENTAGE': ['min', 'max', 'mean', 'sum'],
        'DRAWINGS_POS_PERCENTAGE': ['min', 'max', 'mean', 'sum'],
        'DRAWINGS_OTHER_PERCENTAGE': ['min', 'max', 'mean', 'sum'],
        'FLAG_LATE_PAYMENT': 'mean',                           
        'LIMIT_RATIO': 'mean',                           
    }).reset_index()

    # Flatten kolom multiindex
    agg_ccb.columns = ['CCB_' + '_'.join(col).upper() for col in agg_ccb.columns]

    return agg_ccb

In [86]:
ccb = aggregate_ccb(ccb)

In [87]:
# ccb.columns = ['CCB_' + ''.join(col).upper() for col in ccb.columns]

In [88]:
ccb.head()

,CCB_SK_ID_CURR_,CCB_MONTHS_BALANCE_COUNT,CCB_AMT_BALANCE_MIN,CCB_AMT_BALANCE_MAX,CCB_AMT_BALANCE_MEAN,CCB_AMT_BALANCE_SUM,CCB_AMT_CREDIT_LIMIT_ACTUAL_MIN,CCB_AMT_CREDIT_LIMIT_ACTUAL_MAX,CCB_AMT_CREDIT_LIMIT_ACTUAL_MEAN,CCB_AMT_CREDIT_LIMIT_ACTUAL_SUM,CCB_AMT_DRAWINGS_ATM_CURRENT_MIN,CCB_AMT_DRAWINGS_ATM_CURRENT_MAX,CCB_AMT_DRAWINGS_ATM_CURRENT_MEAN,CCB_AMT_DRAWINGS_ATM_CURRENT_SUM,CCB_AMT_DRAWINGS_CURRENT_MIN,CCB_AMT_DRAWINGS_CURRENT_MAX,CCB_AMT_DRAWINGS_CURRENT_MEAN,CCB_AMT_DRAWINGS_CURRENT_SUM,CCB_AMT_DRAWINGS_OTHER_CURRENT_MIN,CCB_AMT_DRAWINGS_OTHER_CURRENT_MAX,CCB_AMT_DRAWINGS_OTHER_CURRENT_MEAN,CCB_AMT_DRAWINGS_OTHER_CURRENT_SUM,CCB_AMT_INST_MIN_REGULARITY_MIN,CCB_AMT_INST_MIN_REGULARITY_MAX,CCB_AMT_INST_MIN_REGULARITY_MEAN,CCB_AMT_INST_MIN_REGULARITY_SUM,CCB_AMT_PAYMENT_TOTAL_CURRENT_MIN,CCB_AMT_PAYMENT_TOTAL_CURRENT_MAX,CCB_AMT_PAYMENT_TOTAL_CURRENT_MEAN,CCB_AMT_PAYMENT_TOTAL_CURRENT_SUM,CCB_AMT_RECEIVABLE_PRINCIPAL_MIN,CCB_AMT_RECEIVABLE_PRINCIPAL_MAX,CCB_AMT_RECEIVABLE_PRINCIPAL_MEAN,CCB_AMT_RECEIVABLE_PRINCIPAL_SUM,CCB_AMT_RECIVABLE_MIN,CCB_AMT_RECIVABLE_MAX,CCB_AMT_RECIVABLE_MEAN,CCB_AMT_RECIVABLE_SUM,CCB_AMT_TOTAL_RECEIVABLE_MIN,CCB_AMT_TOTAL_RECEIVABLE_MAX,CCB_AMT_TOTAL_RECEIVABLE_MEAN,CCB_AMT_TOTAL_RECEIVABLE_SUM,CCB_CNT_DRAWINGS_ATM_CURRENT_MIN,CCB_CNT_DRAWINGS_ATM_CURRENT_MAX,CCB_CNT_DRAWINGS_ATM_CURRENT_MEAN,CCB_CNT_DRAWINGS_ATM_CURRENT_SUM,CCB_AMT_DRAWINGS_POS_CURRENT_MIN,CCB_AMT_DRAWINGS_POS_CURRENT_MAX,CCB_AMT_DRAWINGS_POS_CURRENT_MEAN,CCB_AMT_DRAWINGS_POS_CURRENT_SUM,CCB_AMT_PAYMENT_CURRENT_MIN,CCB_AMT_PAYMENT_CURRENT_MAX,CCB_AMT_PAYMENT_CURRENT_MEAN,CCB_AMT_PAYMENT_CURRENT_SUM,CCB_CNT_DRAWINGS_OTHER_CURRENT_MIN,CCB_CNT_DRAWINGS_OTHER_CURRENT_MAX,CCB_CNT_DRAWINGS_OTHER_CURRENT_MEAN,CCB_CNT_DRAWINGS_OTHER_CURRENT_SUM,CCB_CNT_DRAWINGS_CURRENT_MIN,CCB_CNT_DRAWINGS_CURRENT_MAX,CCB_CNT_DRAWINGS_CURRENT_MEAN,CCB_CNT_DRAWINGS_CURRENT_SUM,CCB_CNT_DRAWINGS_POS_CURRENT_MIN,CCB_CNT_DRAWINGS_POS_CURRENT_MAX,CCB_CNT_DRAWINGS_POS_CURRENT_MEAN,CCB_CNT_DRAWINGS_POS_CURRENT_SUM,CCB_CNT_INSTALMENT_MATURE_CUM_MIN,CCB_CNT_INSTALMENT_MATURE_CUM_MAX,CCB_CNT_INSTALMENT_MATURE_CUM_MEAN,CCB_CNT_INSTALMENT_MATURE_CUM_SUM,CCB_SK_DPD_MIN,CCB_SK_DPD_MAX,CCB_SK_DPD_MEAN,CCB_SK_DPD_SUM,CCB_SK_DPD_DEF_MIN,CCB_SK_DPD_DEF_MAX,CCB_SK_DPD_DEF_MEAN,CCB_SK_DPD_DEF_SUM,CCB_TOTAL_DRAWINGS_MIN,CCB_TOTAL_DRAWINGS_MAX,CCB_TOTAL_DRAWINGS_MEAN,CCB_TOTAL_DRAWINGS_SUM,CCB_TOTAL_DRAWINGS_COUNT_MIN,CCB_TOTAL_DRAWINGS_COUNT_MAX,CCB_TOTAL_DRAWINGS_COUNT_MEAN,CCB_TOTAL_DRAWINGS_COUNT_SUM,CCB_AVG_DRAWING_MIN,CCB_AVG_DRAWING_MAX,CCB_AVG_DRAWING_MEAN,CCB_AVG_DRAWING_SUM,CCB_DRAWINGS_ATM_PERCENTAGE_MIN,CCB_DRAWINGS_ATM_PERCENTAGE_MAX,CCB_DRAWINGS_ATM_PERCENTAGE_MEAN,CCB_DRAWINGS_ATM_PERCENTAGE_SUM,CCB_DRAWINGS_POS_PERCENTAGE_MIN,CCB_DRAWINGS_POS_PERCENTAGE_MAX,CCB_DRAWINGS_POS_PERCENTAGE_MEAN,CCB_DRAWINGS_POS_PERCENTAGE_SUM,CCB_DRAWINGS_OTHER_PERCENTAGE_MIN,CCB_DRAWINGS_OTHER_PERCENTAGE_MAX,CCB_DRAWINGS_OTHER_PERCENTAGE_MEAN,CCB_DRAWINGS_OTHER_PERCENTAGE_SUM,CCB_FLAG_LATE_PAYMENT_MEAN,CCB_LIMIT_RATIO_MEAN
0,100006,6,0.0,0.00000,0.000000,0.00,270000,270000,270000.000000,1620000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000,0.00,0.000000,0.00000,0.000000,0.000,0.000000,0.00000,0.000000,0.000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,2702.699951,2702.699951,2702.699951,16216.199219,0.0,0.0,0.0,0.0,0,0,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0,0,0.000000,0,0,0,0.000000,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.000000
1,100011,74,0.0,189000.00000,54482.113281,4031676.25,90000,180000,164189.189189,12150000,0.0,180000.0,2432.432373,180000.0,0.0,180000.0,2432.432373,180000.0,0.0,0.0,0.0,0.0,0.0,9000.0,3902.759277,288804.187500,0.0,55485.0,4520.067383,334485.0000,0.0,180000.0,52402.085938,3877754.50,-563.354980,189000.00000,54433.179688,4028055.250,-563.354980,189000.00000,54433.179688,4028055.250,0.0,4.0,0.05405

In [89]:
ccb_clean = ccb.copy()

In [90]:
print('Train dimension:', train_clean.shape)
print('Test dimension:', test_clean.shape)
print('Bureau & Bureau Balance dimension:', bureau_clean.shape)
print('Instalment dimension: ', installment_clean.shape)
print('Credit Card Balance dimension: ', ccb_clean.shape)

Train dimension: (307511, 89)
Test dimension: (48744, 88)
Bureau & Bureau Balance dimension: (305811, 41)
Instalment dimension:  (339587, 33)
Credit Card Balance dimension:  (103558, 104)


## 3.3 Pos Cash Balance

In [91]:
reduce_memory_usage(pos_cb)
pos_cb.info()

Memory usage of dataframe is 610.43 MB
Memory usage after optimization is: 238.45 MB
Decreased by 60.9%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   SK_ID_PREV             int32  
 1   SK_ID_CURR             int32  
 2   MONTHS_BALANCE         int8   
 3   CNT_INSTALMENT         float16
 4   CNT_INSTALMENT_FUTURE  float16
 5   NAME_CONTRACT_STATUS   object 
 6   SK_DPD                 int16  
 7   SK_DPD_DEF             int16  
dtypes: float16(2), int16(2), int32(2), int8(1), object(1)
memory usage: 238.5+ MB


In [92]:
pos_cb = replace_missing_with_null(pos_cb, missing_values)

In [93]:
# Check Null Value
null_analysis = check_null_percentage(pos_cb)
print("Total rows in dataset:", len(pos_cb))
print("\nNull Value Analysis:")
print(null_analysis)

Total rows in dataset: 10001358

Null Value Analysis:
                       Total_Null  Null_Percentage
CNT_INSTALMENT              26071             0.26
CNT_INSTALMENT_FUTURE       26087             0.26
NAME_CONTRACT_STATUS            2             0.00


In [94]:
def handling_na_ccb(data):
    # Mengisi nilai null dengan median
    columns_fill_median = ['CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE']
    
    data[columns_fill_median] = data[columns_fill_median].fillna(data[columns_fill_median].median())

    # Mengisi nilai null dengan modus
    columns_fill_mode = ['NAME_CONTRACT_STATUS']

    for column in columns_fill_mode:
        mode_value = data[column].mode()[0]
        data[column] = data[column].fillna(mode_value)

In [95]:
handling_na_ccb(pos_cb)

In [96]:
pos_cb.duplicated().sum()

np.int64(0)

In [97]:
# Feature Engineering
def pos_cash_balance_FE(data):
    
    # Menghitung Ratio Cicilan yang Berjalan
    data['PAYMENT_PROGRESS_RATIO'] = data['CNT_INSTALMENT'] / data['CNT_INSTALMENT_FUTURE']

    # Menghitung Sisa Cicilan
    data['REMAINING_PAYMENT'] = data['CNT_INSTALMENT_FUTURE'] - data['CNT_INSTALMENT']


In [98]:
pos_cash_balance_FE(pos_cb)

In [99]:
column_obj = pos_cb.select_dtypes('object').columns
pos_cb = pos_cb.drop(columns=column_obj)

In [100]:
def aggregate_pos_cash(df, group_by='SK_ID_CURR'):
    agg_pos = df.groupby(group_by).agg({
        # Jumlah record per customer
        'SK_ID_PREV': 'nunique',  # Menghitung jumlah unique previous applications
        
        # Agregasi MONTHS_BALANCE
        'MONTHS_BALANCE': ['min', 'max', 'mean', 'size'],  # size untuk menghitung total records
        
        # Agregasi installment counts
        'CNT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
        'CNT_INSTALMENT_FUTURE': ['min', 'max', 'mean', 'sum'],
        
        # Agregasi untuk status kontrak (categorical)
        #'NAME_CONTRACT_STATUS': lambda x: dict(x.value_counts()), # Menghitung frekuensi setiap status
        
        # Agregasi untuk SK_DPD (Days Past Due)
        'SK_DPD': ['max', 'mean', 'sum'],
        'SK_DPD_DEF': ['max', 'mean', 'sum']
    })
    
    # Flatten kolom multiindex
    agg_pos.columns = ['POS_' + '_'.join(col).upper() for col in agg_pos.columns.values]
    
    return agg_pos

In [101]:
#pos_cb.columns = ['Pos_' + ''.join(col).upper() for col in pos_cb.columns]

In [102]:
posh_cb_clean = aggregate_pos_cash(pos_cb, group_by='SK_ID_CURR')

In [103]:
posh_cb_clean.head()

,POS_SK_ID_PREV_NUNIQUE,POS_MONTHS_BALANCE_MIN,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_CNT_INSTALMENT_MIN,POS_CNT_INSTALMENT_MAX,POS_CNT_INSTALMENT_MEAN,POS_CNT_INSTALMENT_SUM,POS_CNT_INSTALMENT_FUTURE_MIN,POS_CNT_INSTALMENT_FUTURE_MAX,POS_CNT_INSTALMENT_FUTURE_MEAN,POS_CNT_INSTALMENT_FUTURE_SUM,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_SUM,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_SK_DPD_DEF_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,
100001,2,-96,-53,-72.555556,9,4.0,4.0,4.000000,36.0,0.0,4.0,1.444444,13.0,7,0.777778,7,7,0.777778,7
100002,1,-19,-1,-10.000000,19,24.0,24.0,24.000000,456.0,6.0,24.0,15.000000,285.0,0,0.000000,0,0,0.000000,0
100003,3,-77,-18,-43.785714,28,6.0,12.0,10.107142,283.0,0.0,12.0,5.785714,162.0,0,0.000000,0,0,0.000000,0
100004,1,-27,-24,-25.500000,4,3.0,4.0,3.750000,15.0,0.0,4.0,2.250000,9.0,0,0.000000,0,0,0.000000,0
100005,1,-25,-15,-20.000000,11,9.0,12.0,11.727273,129.0,0.0,12.0,7.181818,79.0,0,0.000000,0,0,0.000000,0


In [104]:
print('Train dimension:', train_clean.shape)
print('Test dimension:', test_clean.shape)
print('Bureau & Bureau Balance dimension:', bureau_clean.shape)
print('Instalment dimension: ', installment_clean.shape)
print('Credit Card Balance dimension: ', ccb_clean.shape)
print('Pos Cash Balance dimension: ', posh_cb_clean.shape)

Train dimension: (307511, 89)
Test dimension: (48744, 88)
Bureau & Bureau Balance dimension: (305811, 41)
Instalment dimension:  (339587, 33)
Credit Card Balance dimension:  (103558, 104)
Pos Cash Balance dimension:  (337252, 19)


## 3.4 Previous Application


In [105]:
reduce_memory_usage(prev)
prev.info()

Memory usage of dataframe is 471.48 MB
Memory usage after optimization is: 309.01 MB
Decreased by 34.5%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int32  
 1   SK_ID_CURR                   1670214 non-null  int32  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float32
 4   AMT_APPLICATION              1670214 non-null  float32
 5   AMT_CREDIT                   1670213 non-null  float32
 6   AMT_DOWN_PAYMENT             774370 non-null   float32
 7   AMT_GOODS_PRICE              1284699 non-null  float32
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int8   
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11

In [106]:
prev = replace_missing_with_null(prev, missing_values)

In [107]:
# Check Null Value
null_analysis = check_null_percentage(prev)
print("Total rows in dataset:", len(prev))
print("\nNull Value Analysis:")
print(null_analysis)

Total rows in dataset: 1670214

Null Value Analysis:
                           Total_Null  Null_Percentage
RATE_INTEREST_PRIVILEGED      1664263            99.64
RATE_INTEREST_PRIMARY         1664263            99.64
NAME_PRODUCT_TYPE             1063666            63.68
NAME_GOODS_CATEGORY            950809            56.93
AMT_DOWN_PAYMENT               895844            53.64
RATE_DOWN_PAYMENT              895844            53.64
NAME_SELLER_INDUSTRY           855720            51.23
NAME_TYPE_SUITE                820405            49.12
NAME_CASH_LOAN_PURPOSE         677918            40.59
DAYS_TERMINATION               673065            40.30
DAYS_FIRST_DUE                 673065            40.30
DAYS_LAST_DUE                  673065            40.30
DAYS_LAST_DUE_1ST_VERSION      673065            40.30
NFLAG_INSURED_ON_APPROVAL      673065            40.30
DAYS_FIRST_DRAWING             673065            40.30
NAME_PAYMENT_TYPE              627384            37.56
NAME_YIELD_G

Terdapat 2 features yang memiliki Null Percentage sebesar 99% sehingga feature tidak memiliki arti untuk model dan akan dilakukan drop features dan sisanya akan diisi dengan median dan unknown untuk null value di kisaran 20% - 65% dan modus untuk di bawah 20%.

In [108]:
prev['NAME_CONTRACT_TYPE'].unique()

array(['Consumer loans', 'Cash loans', 'Revolving loans', nan],
      dtype=object)

In [109]:
prev = prev.drop(columns=['RATE_INTEREST_PRIVILEGED', 'RATE_INTEREST_PRIMARY', 'DAYS_LAST_DUE_1ST_VERSION'], axis = 1)

In [110]:
def handling_na_prev(data):

    # Mengisi nilai null dengan median
    columns_fill_median = ['AMT_DOWN_PAYMENT', 'RATE_DOWN_PAYMENT', 
                            'DAYS_TERMINATION', 'DAYS_FIRST_DUE', 
                            'DAYS_LAST_DUE', 'NFLAG_INSURED_ON_APPROVAL', 
                            'DAYS_FIRST_DRAWING', 'AMT_GOODS_PRICE', 
                            'AMT_ANNUITY', 'CNT_PAYMENT', 'AMT_CREDIT']
    
    data[columns_fill_median] = data[columns_fill_median].fillna(data[columns_fill_median].median())
    
    # Mengisi nilai null dengan unknown
    columns_fill_unknown = ['NAME_PRODUCT_TYPE', 'NAME_GOODS_CATEGORY', 'NAME_SELLER_INDUSTRY', 'NAME_TYPE_SUITE',
                            'NAME_CASH_LOAN_PURPOSE', 'NAME_PAYMENT_TYPE', 'NAME_YIELD_GROUP', 'NAME_PORTFOLIO']
    
    data[columns_fill_unknown] = data[columns_fill_unknown].fillna('unknown')

    # Mengisi nilai null dengan modus
    columns_fill_mode = ['CODE_REJECT_REASON', 'NAME_CLIENT_TYPE', 'PRODUCT_COMBINATION', 'NAME_CONTRACT_TYPE']

    for column in columns_fill_mode:
        mode_value = data[column].mode()[0]
        data[column] = data[column].fillna(mode_value)


In [111]:
handling_na_prev(prev)

In [112]:
prev.duplicated().sum()

np.int64(0)

### Label Encoding

In [113]:
prev['NAME_CASH_LOAN_PURPOSE'].value_counts() / sum(prev['NAME_GOODS_CATEGORY'].value_counts())*100

NAME_CASH_LOAN_PURPOSE
XAP                                 55.242083
unknown                             40.588691
Repairs                              1.422872
Other                                0.934491
Urgent needs                         0.503648
Buying a used car                    0.172912
Building a house or an annex         0.161237
Everyday expenses                    0.144652
Medicine                             0.130163
Payments on other loans              0.115614
Education                            0.094180
Journey                              0.074182
Purchase of electronic equipment     0.063525
Buying a new car                     0.060591
Wedding / gift / holiday             0.057597
Buying a home                        0.051790
Car repairs                          0.047718
Furniture                            0.044845
Buying a holiday home / land         0.031912
Business development                 0.025506
Gasification / water supply          0.017962
Buying a ga

Menggabungkan Unknown dengan XAP dan value yang berada di bawah 0.1% akan masuk ke Other.

In [114]:
prev['NAME_GOODS_CATEGORY'].value_counts() / sum(prev['NAME_GOODS_CATEGORY'].value_counts())*100

NAME_GOODS_CATEGORY
unknown                     56.927376
Mobile                      13.453845
Consumer Electronics         7.279067
Computers                    6.332662
Audio/Video                  5.953788
Furniture                    3.212522
Photo / Cinema Equipment     1.498072
Construction Materials       1.496515
Clothing and Accessories     1.410238
Auto Accessories             0.441919
Jewelry                      0.376598
Homewares                    0.300740
Medical Supplies             0.230090
Vehicles                     0.201771
Sport and Leisure            0.178480
Gardening                    0.159740
Other                        0.152915
Office Appliances            0.139683
Tourism                      0.099329
Medicine                     0.092802
Direct Sales                 0.026703
Fitness                      0.012513
Additional Service           0.007664
Education                    0.006406
Weapon                       0.004610
Insurance                    0

In [115]:
prev.select_dtypes('object').columns

Index(['NAME_CONTRACT_TYPE', 'WEEKDAY_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'NAME_PAYMENT_TYPE', 'CODE_REJECT_REASON',
       'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE', 'NAME_GOODS_CATEGORY',
       'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE', 'CHANNEL_TYPE',
       'NAME_SELLER_INDUSTRY', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION'],
      dtype='object')

In [116]:
prev['NAME_CASH_LOAN_PURPOSE'].value_counts() / sum(prev['NAME_GOODS_CATEGORY'].value_counts())*100

NAME_CASH_LOAN_PURPOSE
XAP                                 55.242083
unknown                             40.588691
Repairs                              1.422872
Other                                0.934491
Urgent needs                         0.503648
Buying a used car                    0.172912
Building a house or an annex         0.161237
Everyday expenses                    0.144652
Medicine                             0.130163
Payments on other loans              0.115614
Education                            0.094180
Journey                              0.074182
Purchase of electronic equipment     0.063525
Buying a new car                     0.060591
Wedding / gift / holiday             0.057597
Buying a home                        0.051790
Car repairs                          0.047718
Furniture                            0.044845
Buying a holiday home / land         0.031912
Business development                 0.025506
Gasification / water supply          0.017962
Buying a ga

In [117]:
# Feature Encoding
def prev_encoding(data):
    data['WEEKDAY_APPR_PROCESS_START'] = data['WEEKDAY_APPR_PROCESS_START'].map({'MONDAY': 1, 'TUESDAY': 2, 'WEDNESDAY': 3, 'THURSDAY': 4, 'FRIDAY': 5, 'SATURDAY': 6, 'SUNDAY': 7})
    data['FLAG_LAST_APPL_PER_CONTRACT'] = data['FLAG_LAST_APPL_PER_CONTRACT'].map({'N': 0, 'Y': 1})
    data['NAME_CASH_LOAN_PURPOSE'] = data['NAME_CASH_LOAN_PURPOSE'].replace({'unknown': 'XAP'}) # Menggabungkan unknown dengan XAP
    data['NAME_YIELD_GROUP'] = data['NAME_YIELD_GROUP'].map({'unknown': 0, 'low_action': 1, 'low_normal': 2, 'middle': 3, 'high': 4})
    prev['NAME_CASH_LOAN_PURPOSE'] = prev['NAME_CASH_LOAN_PURPOSE'].replace({
                                                                    'Education': 'Other', 'Journey': 'Other', 'Purchase of electronic equipment': 'Other', 'Buying a new car': 'Other',
                                                                    'Wedding / gift / holiday': 'Other', 'Buying a home': 'Other', 'Car repairs': 'Other', 'Furniture': 'Other',
                                                                    'Buying a holiday home / land': 'Other', 'Business development': 'Other', 'Gasification / water supply': 'Other',
                                                                    'Buying a garage': 'Other', 'Hobby': 'Other', 'Money for a third person': 'Other', 'Refusal to name the goal': 'Other',
    })
    data['NAME_GOODS_CATEGORY'] = data['NAME_GOODS_CATEGORY'].map({
                                                                    # Electronics group
                                                                    'Mobile': 'Electronics',
                                                                    'Consumer Electronics': 'Electronics',
                                                                    'Computers': 'Electronics',
                                                                    'Audio/Video': 'Electronics',
                                                                    'Photo / Cinema Equipment': 'Electronics',
                                                                    'Office Appliances': 'Electronics',
                                                                    
                                                                    # Home and Living group
                                                                    'Furniture': 'Home and Living',
                                                                    'Construction Materials': 'Home and Living',
                                                                    'Homewares': 'Home and Living',
                                                                    'Gardening': 'Home and Living',
                                                                    'House Construction': 'Home and Living',
                                                                    
                                                                    # Fashion and Accessories group
                                                                    'Clothing and Accessories': 'Fashion',
                                                                    'Jewelry': 'Fashion',
                                                                    
                                                                    # Auto and Transport group
                                                                    'Auto Accessories': 'Auto and Transport',
                                                                    'Vehicles': 'Auto and Transport',
                                                                    
                                                                    # Health and Wellness group
                                                                    'Medical Supplies': 'Health and Wellness',
                                                                    'Medicine': 'Health and Wellness',
                                                                    'Fitness': 'Health and Wellness',
                                                                    'Sport and Leisure': 'Health and Wellness',
                                                                    
                                                                    # Services group
                                                                    'Tourism': 'Services',
                                                                    'Direct Sales': 'Services',
                                                                    'Additional Service': 'Services',
                                                                    'Education': 'Services',
                                                                    'Insurance': 'Services',
                                                                    
                                                                    # Others
                                                                    'Other': 'Other',
                                                                    'unknown': 'Other',
                                                                    'Animals': 'Other',
                                                                    'Weapon': 'Other'
                                                                })
    

In [118]:
prev_encoding(prev)

In [119]:
prev[['WEEKDAY_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT', 'NAME_CASH_LOAN_PURPOSE', 'NAME_YIELD_GROUP', 'NAME_GOODS_CATEGORY']].sample(5)

,WEEKDAY_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_YIELD_GROUP,NAME_GOODS_CATEGORY
1026093,3,1,XAP,2,Home and Living
121264,4,1,XAP,3,Other
1168302,2,1,XAP,2,Other
432848,4,1,XAP,0,Other
542221,1,1,XAP,0,Other


In [120]:
prev['NAME_CASH_LOAN_PURPOSE'].unique()

array(['XAP', 'Repairs', 'Everyday expenses', 'Other',
       'Building a house or an annex', 'Medicine',
       'Payments on other loans', 'Urgent needs', 'Buying a used car'],
      dtype=object)

In [121]:
prev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 34 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int32  
 1   SK_ID_CURR                   1670214 non-null  int32  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1670214 non-null  float32
 4   AMT_APPLICATION              1670214 non-null  float32
 5   AMT_CREDIT                   1670214 non-null  float32
 6   AMT_DOWN_PAYMENT             1670214 non-null  float32
 7   AMT_GOODS_PRICE              1670214 non-null  float32
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  int64  
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int8   
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  int64  
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int8   
 12  RATE_DOWN_PAYMENT            1670214 non-n

In [122]:
prev['NAME_YIELD_GROUP'].unique()

array([3, 1, 4, 2, 0])

In [123]:
column_obj = prev.select_dtypes('object').columns
prev = prev.drop(columns=column_obj)

In [124]:
def aggregate_previous_application(df, group_by='SK_ID_CURR'):
   agg_prev = df.groupby(group_by).agg({
       # Count unique previous applications
       'SK_ID_PREV': 'nunique',
       
       # Amount aggregations
       'AMT_ANNUITY': ['min', 'max', 'mean', 'sum'],
       'AMT_APPLICATION': ['min', 'max', 'mean', 'sum'],
       'AMT_CREDIT': ['min', 'max', 'mean', 'sum'],
       'AMT_DOWN_PAYMENT': ['min', 'max', 'mean', 'sum'],
       'AMT_GOODS_PRICE': ['min', 'max', 'mean', 'sum'],
       
       # Time-related features
       'WEEKDAY_APPR_PROCESS_START': ['nunique', lambda x: x.mode().iloc[0] if not x.empty else None],
       'HOUR_APPR_PROCESS_START': ['nunique', lambda x: x.mode().iloc[0] if not x.empty else None],
       
       # Flag aggregations
       'FLAG_LAST_APPL_PER_CONTRACT': ['mean', 'sum'],
       'NFLAG_LAST_APPL_IN_DAY': ['mean', 'sum'],
       'NFLAG_INSURED_ON_APPROVAL': ['mean', 'sum'],
       
       # Payment related
       'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
       'CNT_PAYMENT': ['min', 'max', 'mean', 'sum'],
       
       # Days related
       'DAYS_DECISION': ['min', 'max', 'mean'],
       'DAYS_FIRST_DRAWING': ['min', 'max', 'mean'],
       'DAYS_FIRST_DUE': ['min', 'max', 'mean'],
       'DAYS_LAST_DUE': ['min', 'max', 'mean'],
       'DAYS_TERMINATION': ['min', 'max', 'mean'],
       
       # Other features
       'SELLERPLACE_AREA': ['nunique', 'mean'],
       'NAME_YIELD_GROUP': 'nunique'
   })
   
   # Flatten columns
   agg_prev.columns = ['PREV_' + '_'.join(str(col) for col in col_name).upper() for col_name in agg_prev.columns]
   
   return agg_prev 

In [125]:
prev_clean = aggregate_previous_application(prev, group_by='SK_ID_CURR')

In [126]:
#prev.columns = ['Prev_' + ''.join(col).upper() for col in prev.columns]

In [127]:
prev_clean.sample()

,PREV_SK_ID_PREV_NUNIQUE,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_ANNUITY_SUM,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_MEAN,PREV_AMT_APPLICATION_SUM,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_MEAN,PREV_AMT_CREDIT_SUM,PREV_AMT_DOWN_PAYMENT_MIN,PREV_AMT_DOWN_PAYMENT_MAX,PREV_AMT_DOWN_PAYMENT_MEAN,PREV_AMT_DOWN_PAYMENT_SUM,PREV_AMT_GOODS_PRICE_MIN,PREV_AMT_GOODS_PRICE_MAX,PREV_AMT_GOODS_PRICE_MEAN,PREV_AMT_GOODS_PRICE_SUM,PREV_WEEKDAY_APPR_PROCESS_START_NUNIQUE,PREV_WEEKDAY_APPR_PROCESS_START_<LAMBDA_0>,PREV_HOUR_APPR_PROCESS_START_NUNIQUE,PREV_HOUR_APPR_PROCESS_START_<LAMBDA_0>,PREV_FLAG_LAST_APPL_PER_CONTRACT_MEAN,PREV_FLAG_LAST_APPL_PER_CONTRACT_SUM,PREV_NFLAG_LAST_APPL_IN_DAY_MEAN,PREV_NFLAG_LAST_APPL_IN_DAY_SUM,PREV_NFLAG_INSURED_ON_APPROVAL_MEAN,PREV_NFLAG_INSURED_ON_APPROVAL_SUM,PREV_RATE_DOWN_PAYMENT_MIN,PREV_RATE_DOWN_PAYMENT_MAX,PREV_RATE_DOWN_PAYMENT_MEAN,PREV_CNT_PAYMENT_MIN,PREV_CNT_PAYMENT_MAX,PREV_CNT_PAYMENT_MEAN,PREV_CNT_PAYMENT_SUM,PREV_DAYS_DECISION_MIN,PREV_DAYS_DECISION_MAX,PREV_DAYS_DECISION_MEAN,PREV_DAYS_FIRST_DRAWING_MIN,PREV_DAYS_FIRST_DRAWING_MAX,PREV_DAYS_FIRST_DRAWING_MEAN,PREV_DAYS_FIRST_DUE_MIN,PREV_DAYS_FIRST_DUE_MAX,PREV_DAYS_FIRST_DUE_MEAN,PREV_DAYS_LAST_DUE_MIN,PREV_DAYS_LAST_DUE_MAX,PREV_DAYS_LAST_DUE_MEAN,PREV_DAYS_TERMINATION_MIN,PREV_DAYS_TERMINATION_MAX,PREV_DAYS_TERMINATION_MEAN,PREV_SELLERPLACE_AREA_NUNIQUE,PREV_SELLERPLACE_AREA_MEAN,PREV_NAME_YIELD_GROUP_NUNIQUE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
123712,10,9170.549805,26901.0,17286.746094,172867.453125,0.0,810000.0,206374.5,2063745.0,46485.0,810000.0,240919.65625,2409196.5,0.0,10174.5,1836.449951,18364.5,45000.0,810000.0,217606.5,2176065.0,4,1,6,8,1.0,10,1.0,10,0.5,5.0,0.0,0.108887,0.036691,0.0,48.0,14.2,142.0,-2397,-429,-1467.4,365243.0,365243.0,365243.0,-2362.0,365243.0,35113.699219,-2092.0,365243.0,35353.101562,-2090.0,365243.0,35362.398438,2,9.1,3


In [128]:
print('Train dimension:', train_clean.shape)
print('Test dimension:', test_clean.shape)
print('Bureau & Bureau Balance dimension:', bureau_clean.shape)
print('Instalment dimension: ', installment_clean.shape)
print('Credit Card Balance dimension: ', ccb_clean.shape)
print('Pos Cash Balance dimension: ', posh_cb_clean.shape)
print('Previous Application dimension: ', prev_clean.shape)

Train dimension: (307511, 89)
Test dimension: (48744, 88)
Bureau & Bureau Balance dimension: (305811, 41)
Instalment dimension:  (339587, 33)
Credit Card Balance dimension:  (103558, 104)
Pos Cash Balance dimension:  (337252, 19)
Previous Application dimension:  (338857, 56)


# Merging Dataset

In [134]:
bureau_clean.head(1)

,BUR_SK_ID_BUREAU_COUNT,BUR_DAYS_CREDIT_MIN,BUR_DAYS_CREDIT_MAX,BUR_DAYS_CREDIT_MEAN,BUR_DAYS_CREDIT_ENDDATE_MIN,BUR_DAYS_CREDIT_ENDDATE_MAX,BUR_DAYS_CREDIT_ENDDATE_MEAN,BUR_DAYS_ENDDATE_FACT_MIN,BUR_DAYS_ENDDATE_FACT_MAX,BUR_DAYS_ENDDATE_FACT_MEAN,BUR_DAYS_CREDIT_UPDATE_MIN,BUR_DAYS_CREDIT_UPDATE_MAX,BUR_DAYS_CREDIT_UPDATE_MEAN,BUR_DAYS_DURATION_CREDIT_MIN,BUR_DAYS_DURATION_CREDIT_MAX,BUR_DAYS_DURATION_CREDIT_MEAN,BUR_CREDIT_DAY_OVERDUE_MAX,BUR_CREDIT_DAY_OVERDUE_MEAN,BUR_CREDIT_DAY_OVERDUE_SUM,BUR_AMT_CREDIT_MAX_OVERDUE_MAX,BUR_AMT_CREDIT_MAX_OVERDUE_MEAN,BUR_AMT_CREDIT_SUM_MAX,BUR_AMT_CREDIT_SUM_MEAN,BUR_AMT_CREDIT_SUM_SUM,BUR_AMT_CREDIT_SUM_DEBT_MAX,BUR_AMT_CREDIT_SUM_DEBT_MEAN,BUR_AMT_CREDIT_SUM_DEBT_SUM,BUR_AMT_CREDIT_SUM_LIMIT_MAX,BUR_AMT_CREDIT_SUM_LIMIT_MEAN,BUR_AMT_CREDIT_SUM_LIMIT_SUM,BUR_AMT_CREDIT_SUM_OVERDUE_MAX,BUR_AMT_CREDIT_SUM_OVERDUE_MEAN,BUR_AMT_CREDIT_SUM_OVERDUE_SUM,BUR_AMT_ANNUITY_MAX,BUR_AMT_ANNUITY_MEAN,BUR_AMT_ANNUITY_SUM,BUR_CNT_CREDIT_PROLONG_MAX,BUR_CNT_CREDIT_PROLONG_SUM,BUR_MONTHS_BALANCE_MIN,BUR_MONTHS_BALANCE_MAX,BUR_MONTHS_BALANCE_MEAN
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,17,-1572,-49,-813.941176,-1329.0,1778.0,-40.588234,-1328.0,-544.0,-851.588257,-155,-6,-103.823529,-859.0,335.0,-37.64706,0,0.0,0,0.0,0.0,378000.0,197981.46875,3365685.0,373239.0,70368.085938,1196257.5,0.0,0.0,0.0,0.0,0.0,0.0,10822.5,2641.235352,44901.0,0,0,-47.714286,0.0,-17.248599


In [139]:
merge1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103285 entries, 0 to 103284
Data columns (total 137 columns):
 #    Column                               Dtype  
---   ------                               -----  
 0    INST_NUM_INSTALMENT_VERSION_NUNIQUE  int64  
 1    INST_NUM_INSTALMENT_VERSION_MEAN     float32
 2    INST_NUM_INSTALMENT_NUMBER_MIN       int16  
 3    INST_NUM_INSTALMENT_NUMBER_MAX       int16  
 4    INST_NUM_INSTALMENT_NUMBER_MEAN      float64
 5    INST_NUM_INSTALMENT_NUMBER_SUM       int64  
 6    INST_DAYS_LATE_MEAN                  float32
 7    INST_DAYS_LATE_MAX                   float16
 8    INST_DAYS_LATE_SUM                   float32
 9    INST_DAYS_LATE_COUNT                 int64  
 10   INST_FLAG_LATE_PAYMENT_MEAN          float64
 11   INST_FLAG_LATE_PAYMENT_SUM           int64  
 12   INST_PAYMENT_RATIO_MEAN              float32
 13   INST_PAYMENT_RATIO_MIN               float32
 14   INST_PAYMENT_DIFF_MEAN               float32
 15   INST_PAYMENT_DI

In [145]:
merge1 = pd.merge(installment_clean, ccb_clean, left_on='SK_ID_CURR', right_on='CCB_SK_ID_CURR_', how='inner')
merge2 = pd.merge(merge1, posh_cb_clean, left_on='CCB_SK_ID_CURR_', right_on='SK_ID_CURR', how='inner')
merge3 = pd.merge(merge2, prev_clean, left_on='CCB_SK_ID_CURR_', right_on='SK_ID_CURR', how='inner')
merge_train1 = pd.merge(train_clean, bureau_clean, on='SK_ID_CURR', how='left')
merge_train_final = pd.merge(merge_train1, merge3, left_on='SK_ID_CURR', right_on='CCB_SK_ID_CURR_', how='left')
merge_test1 = pd.merge(test_clean, bureau_clean, on='SK_ID_CURR', how='left')
merge_test_final = pd.merge(merge_test1, merge3, left_on='SK_ID_CURR', right_on='CCB_SK_ID_CURR_', how='left')

In [146]:
merge_train_final.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 342 columns):
 #    Column                                      Dtype  
---   ------                                      -----  
 0    SK_ID_CURR                                  int32  
 1    TARGET                                      int8   
 2    FLAG_OWN_CAR                                int64  
 3    FLAG_OWN_REALTY                             int64  
 4    CNT_CHILDREN                                int8   
 5    AMT_INCOME_TOTAL                            float32
 6    AMT_CREDIT                                  float32
 7    AMT_ANNUITY                                 float32
 8    AMT_GOODS_PRICE                             float32
 9    NAME_EDUCATION_TYPE                         int64  
 10   REGION_POPULATION_RELATIVE                  float16
 11   DAYS_BIRTH                                  int16  
 12   DAYS_EMPLOYED                               int32  
 13   DAYS_REGISTR

In [147]:
merge_test_final.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Data columns (total 341 columns):
 #    Column                                      Dtype  
---   ------                                      -----  
 0    SK_ID_CURR                                  int32  
 1    FLAG_OWN_CAR                                int64  
 2    FLAG_OWN_REALTY                             int64  
 3    CNT_CHILDREN                                int8   
 4    AMT_INCOME_TOTAL                            float32
 5    AMT_CREDIT                                  float32
 6    AMT_ANNUITY                                 float32
 7    AMT_GOODS_PRICE                             float32
 8    NAME_EDUCATION_TYPE                         int64  
 9    REGION_POPULATION_RELATIVE                  float16
 10   DAYS_BIRTH                                  int16  
 11   DAYS_EMPLOYED                               int32  
 12   DAYS_REGISTRATION                           float16
 13   DAYS_ID_PUBLIS

# Save Dataset

In [148]:
# Ekspor dan kompres data train ke CSV dengan format gzip
merge_train_final.to_csv('new_train.csv.gz', index=False, compression='gzip')
# Ekspor dan kompres data test ke CSV dengan format gzip
merge_test_final.to_csv('new_test.csv.gz', index=False, compression='gzip')


# PADA MERGE NANTI COBA UNTUK PAKAI OUTER KARENA:
1. Nasabah mungkin punya installment tapi tidak punya credit card
2. Nasabah mungkin punya posh cash balance tapi tidak dengan installment